In [ ]:
import requests
from bs4 import BeautifulSoup
import json

link = 'http://m.endic.naver.com/example.nhn?sLn=kr&exampleId=%s&webCrawl=0'
link_n = 6540

data = []


for i in range(link_n):
    url = link % (i+1)
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'html.parser')
    korean = str(soup.find_all(class_="text_wrap")[0])[24:-7]
    eng = str(soup.find_all(class_="cp_trans_area")[0])[27:-6]
    if i % 10 == 0:
        print((korean, eng))
        with open('data.txt', 'w') as outfile:
            json.dump(data, outfile)
    data.append((korean, eng))

In [2]:
import json
data = json.load(open("data.txt"))


In [18]:
N = len(data)
print(N)
kor, eng = [], []
for x in data:
    kor.append(x[0])
    _eng = []
    for word in x[1].split():
        word = word.lower()
        punc = None
        if len(word) > 1:
            if ord(word[-1]) < ord('a') or ord(word[-1]) > ord('z'):
                punc = word[-1]
                word = word[:-1]
        _eng.append(word)
        if punc: _eng.append(punc)
    eng.append(_eng)

print(eng[0])

1730
['how', 'are', 'you', 'doing', '?']


In [61]:
dec_wtoi = {}
dec_itow = []
m_dec = 0

enc_wtoi = {}
enc_itow = []
m_enc = 0

l_enc = 0


for words in kor:
    cnt = 0
    l_enc = max(l_enc, len(words))
    for char in words:
        if char not in enc_wtoi:
            enc_wtoi[char] = m_enc
            enc_itow.append(char)
            m_enc += 1
    
enc_wtoi[' '] = m_enc
enc_itow.append(' ')

dec_wtoi['<START>'] = m_dec
dec_itow.append('<START>')
m_dec += 1

l_dec = 0
for words in eng:
    l_dec = max(l_dec, len(words))
    for word in words:
        if word not in dec_wtoi:
            dec_wtoi[word] = m_dec
            dec_itow.append(word)
            m_dec += 1
        
dec_wtoi['<END>'] = m_dec
dec_itow.append('<END>')
m_dec += 1

print ("l_enc: ", l_enc)
print ("l_dec: ", l_dec)
print ("m_enc: ", m_enc)
print ("m_dec: ", m_dec)

l_enc:  45
l_dec:  18
m_enc:  730
m_dec:  1463


In [62]:
import tensorflow as tf
import numpy as np

In [63]:
X_enc_train = m_enc * np.ones((N, l_enc))
X_dec_train = (m_dec - 1) * np.ones((N, l_dec+1))
y_train = (m_dec - 1) * np.ones((N, l_dec+1))
for i, word in enumerate(kor):
    cnt = 0
    for j, char in enumerate(word):
        if char == ' ': continue
        cnt += 1
        X_enc_train[i, l_enc-cnt] = enc_wtoi[char]

for i, words in enumerate(eng):
    X_dec_train[i][0] = 0
    for j, word in enumerate(words):
        X_dec_train[i, j+1] = y_train[i, j] = dec_wtoi[word]

print(X_enc_train)
print(X_dec_train)
print(y_train)

[[730. 730. 730. ...   2.   1.   0.]
 [730. 730. 730. ...  10.   4.   9.]
 [730. 730. 730. ...  19.  18.  17.]
 ...
 [730. 730. 730. ... 170. 224.   0.]
 [730. 730. 730. ...  84.  22.  47.]
 [730. 730. 730. ...  18. 177. 333.]]
[[0.000e+00 1.000e+00 2.000e+00 ... 1.462e+03 1.462e+03 1.462e+03]
 [0.000e+00 6.000e+00 7.000e+00 ... 1.462e+03 1.462e+03 1.462e+03]
 [0.000e+00 1.200e+01 1.300e+01 ... 1.462e+03 1.462e+03 1.462e+03]
 ...
 [0.000e+00 2.240e+02 1.690e+02 ... 1.462e+03 1.462e+03 1.462e+03]
 [0.000e+00 3.950e+02 3.500e+01 ... 1.462e+03 1.462e+03 1.462e+03]
 [0.000e+00 2.450e+02 4.700e+01 ... 1.462e+03 1.462e+03 1.462e+03]]
[[1.000e+00 2.000e+00 3.000e+00 ... 1.462e+03 1.462e+03 1.462e+03]
 [6.000e+00 7.000e+00 8.000e+00 ... 1.462e+03 1.462e+03 1.462e+03]
 [1.200e+01 1.300e+01 1.400e+01 ... 1.462e+03 1.462e+03 1.462e+03]
 ...
 [2.240e+02 1.690e+02 3.000e+00 ... 1.462e+03 1.462e+03 1.462e+03]
 [3.950e+02 3.500e+01 1.188e+03 ... 1.462e+03 1.462e+03 1.462e+03]
 [2.450e+02 4.700e+01 1.

In [136]:
learning_rate = 1e-3
n_hidden = 200
total_epoch = 100

tf.reset_default_graph()

X_enc = tf.placeholder(tf.int32, [None, l_enc])
X_dec = tf.placeholder(tf.int32, [None, None])
y = tf.placeholder(tf.int32, [None, None])
is_training = tf.placeholder(tf.bool)

def model(X_enc, X_dec, is_training):
    keep_prob = tf.cond(is_training, lambda:tf.constant(0.5), lambda:tf.constant(1.0))
    if is_training is not None:
        trainable = True
    else:
        trainable = False
    embedding_enc = tf.get_variable('embedding_enc', initializer=tf.random_uniform([m_enc, n_hidden], -1.0, 1.0), trainable=trainable)
    mt = tf.get_variable('mt', shape=[1, n_hidden], initializer=tf.zeros_initializer(), trainable=False, dtype=tf.float32)
    embedding_enc = tf.concat([embedding_enc, mt], 0, name='concatt')
    embedding_dec = tf.get_variable('embedding_dec', initializer=tf.random_uniform([m_dec, n_hidden], -1.0, 1.0), trainable=trainable)
    after_embedding_enc = tf.cast(tf.nn.embedding_lookup(embedding_enc, X_enc, validate_indices=False), tf.float32)
    after_embedding_dec = tf.cast(tf.nn.embedding_lookup(embedding_dec, X_dec, validate_indices=False), tf.float32)
    
    inputs = tf.layers.conv2d(inputs=tf.reshape(after_embedding_enc, [-1, l_enc, n_hidden, 1]),
                              filters=1,
                              kernel_size=(1, 5),
                              trainable=trainable)
    
    with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, after_embedding_enc, dtype=tf.float32)
    
    with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, after_embedding_dec, initial_state=enc_states, dtype=tf.float32)
    
    out = tf.layers.dense(outputs, n_hidden, activation=None, trainable=trainable)
    out = tf.tensordot(out, embedding_dec, axes=[[2], [1]])
    print(out.get_shape())
    return out

y_out = model(X_enc, X_dec, is_training)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_out, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

(?, ?, 1463)


In [140]:
from IPython.display import clear_output
import random

batch_size = 128
test_num = 200
N_ = N - test_num

indices = np.arange(N)
np.random.shuffle(indices)
test_indices = indices[-test_num:]
train_indices = indices[:-test_num]


sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

stp = 0
best_counter = 0
for epoch in range(1000):
    epoch_indices = np.arange(N_)
    np.random.shuffle(epoch_indices)
    th = 0
    while th < N_:
        rng = train_indices[epoch_indices[th:min(th+batch_size, N_)]]
        feed_dict = {X_enc: X_enc_train[rng],
                     X_dec: X_dec_train[rng],
                     is_training: True,
                     y: y_train[rng]}
        th += batch_size
        loss, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        stp += 1
        
    clear_output()
    print ('Epoch #%d' % (epoch+1))
    print ('loss:', loss)
    y_now = np.zeros((test_num, 1))
    rng = test_indices[:]
    for i in range(1, l_dec):
        feed_dict = {X_enc: X_enc_train[rng],
                     X_dec: X_dec_train[rng],
                     is_training: False}
        res = np.array(sess.run(y_out, feed_dict=feed_dict))
        y_now = np.argmax(res, axis=2)
        
    feed_dict = {X_enc: X_enc_train[rng],
                 X_dec: X_dec_train[rng],
                 y: y_train[rng],
                 is_training: False}
    loss, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
    print ('val_loss:', loss)
    
    correct = 0
    for ii in range(test_num):
        i = test_indices[ii]
        import random
        print_mode = random.random() < 0.025
        if print_mode:
            print (" input: ", kor[i])
            print ("answer: ", " ".join(eng[i]))
            print (" model: ", end=" ")
        model_answer = []
        for j in range(l_dec):
            if y_now[ii, j] == m_dec - 1:
                break
            model_answer.append(dec_itow[y_now[ii, j]])
        model_answer = " ".join(model_answer)
        if model_answer == " ".join(eng[i]):
            correct += 1
        if print_mode:
            print(model_answer, end='\n\n')
    print("correct: ", correct)
    if correct >= best_counter:
        best_counter = correct
        saver.save(sess, "./model.ckpt")
        
    """
    for ii in range(200):
        i = np.random.randint(0, N)
        print (" input: ", kor[i])
        print ("answer: ", " ".join(eng[i]))
        print (" model: ", end=" ")
        y_now = [0]
        model_answer = []
        for j in range(l_dec):
            res = np.array(sess.run(y_out, feed_dict={X_enc: X_enc_train[i:i+1],
                                                      X_dec: np.array([y_now]),
                                                      is_training: False}))
            best_word_idx = np.argmax(res[0][j])
            y_now.append(best_word_idx)
            if best_word_idx == m_dec - 1:
                break
            model_answer.append(dec_itow[np.argmax(res[0][j])])
        model_answer = " ".join(model_answer)
        if model_answer == " ".join(eng[i]):
            model_answer += ' <- and the answer is correct!'
        print(model_answer, end='\n\n')
    """

Epoch #82
loss: 0.10514888
val_loss: 0.038490854
 input:  그밖에 달리 설명할 방법이 없어요.
answer:  i can't explain it in any other way .
 model:  i can't explain it in any other way .

 input:  어서 오십시오.
answer:  hello , may i help you ?
 model:  hello . may i help you ?

 input:  그것은 견해상의 문제입니다.
answer:  it's a matter of opinion .
 model:  it's a matter of opinion .

 input:  그 계획에 찬성합니다.
answer:  i agree with the plan .
 model:  i agree with the plan .

correct:  171


KeyboardInterrupt: 

In [111]:
def do_translate(sentence):
    X_test = m_enc * np.ones((1, l_enc))
    cnt = 0
    for char in sentence:
        if char == ' ' or char not in enc_wtoi: continue
        cnt += 1
        X_test[0, l_enc-cnt] = enc_wtoi[char]
    # print(X_test)
    y_now = [0]
    model_answer = []
    for j in range(l_dec):
        res = np.array(sess.run(y_out, feed_dict={X_enc: X_test,
                                                  X_dec: np.array([y_now]),
                                                  is_training: False}))
        best_word_idx = np.argmax(res[0][j])
        y_now.append(best_word_idx)
        if best_word_idx == m_dec - 1:
            break
        model_answer.append(dec_itow[np.argmax(res[0][j])])
    model_answer = " ".join(model_answer)
    print(model_answer, end='\n\n')

In [124]:
do_translate("")

we walk ?



Epoch #33
loss: 0.5201072
val_loss: 0.38887665
 input:  좋습니다. 참 좋은 생각이에요.
answer:  o.k . that's a good idea .
 model:  that's . i a good idea .

 input:  제 차를 다 고쳤습니까?
answer:  is my car ready to go ?
 model:  is my car to to go ?

 input:  당신 차 좀 사용할 수 있을까요?
answer:  do you think i could use your car ?
 model:  can you i i ask your your car ?

 input:  김씨를 소개하겠습니다.
answer:  may i introduce mr . kim to you ?
 model:  may i introduce . . kim your you ?

 input:  현재 위치를 가르쳐 주세요.
answer:  please point out where i am on this map .
 model:  please get out on i am on this . .

 input:  아주 좋은 날씨야.
answer:  what a beautiful day !
 model:  what a nice day .

correct:  45

Epoch #35
loss: 0.41394392
val_loss: 0.32920885
 input:  제발 나 그만 웃기세요.
answer:  don't make me laugh , please .
 model:  don't make me at . please .

 input:  여름에는 비가 많이 옵니다.
answer:  it rains a lot in summer .
 model:  it rains is lot in summer .

correct:  50

Epoch #36
loss: 0.40963733
val_loss: 0.3046625
 input:  어디가 제일 좋을까요?
answer:  what is the best place to go ?
 model:  what is the best place to go ?

 input:  가능하시다면 편지주세요.
answer:  perhaps you could write to me .
 model:  please you could write to me .

 input:  비가 쏟아질 것 같습니다.
answer:  it threatens to rain .
 model:  it looks to rain .

 input:  여기서 시내로 연결되는 버스가 있습니까?
answer:  is there a bus that goes to downtown from here ?
 model:  is there a bus that downtown to hear from here ?

 input:  당신이 어떻게 나한테 그렇게 말할 수 있어요?
answer:  why could you take to me like that ?
 model:  why could you take your me ? that ?

 input:  시내까지는 몇 정거장을 가야 합니까?
answer:  how many stops are we from downtown ?
 model:  how many stops are we be here ?

 input:  끼여들지 마세요!
answer:  don't cut in !
 model:  don't in in !

 input:  여기서 뉴욕역까진 시간이 얼마나 걸립니까?
answer:  how long will it take to get to new york station from here ?
 model:  how long will it take to get to new york station from here ?

 input:  당신에게 솔직하게 얘기하고 있어요.
answer:  i'm being very honest with you .
 model:  i'm being very honest with you .

correct:  63

poch #38
loss: 0.33438048
val_loss: 0.2540911
 input:  좋습니다. 참 좋은 생각이에요.
answer:  o.k . that's a good idea .
 model:  that's . that's a good idea .

 input:  힘 내시고, 다음 번에 다시 해 보세요.
answer:  come on , try it again .
 model:  come on , the it , .

 input:  무슨 일입니까?
answer:  what's up ?
 model:  what's up ?

 input:  그럼, 다음에 또 뵐게요. 안녕히 계세요.
answer:  well , see you later . good bye .
 model:  well , see you later . good bye .

correct:  69

Epoch #39
loss: 0.38782102
val_loss: 0.23249874
 input:  최소한 사과는 할 수 있잖아요?
answer:  you might at least apologize .
 model:  you might at least apologize .

 input:  발 디딜 틈도 없군요.
answer:  there's no place to put my feet .
 model:  there's no place to put my feet .

 input:  현재 위치를 가르쳐 주세요.
answer:  please point out where i am on this map .
 model:  please point out on i am on this map .

 input:  그게 어쨌단 말이에요?
answer:  what about it ?
 model:  what about it ?

 input:  흥분하지 말아요.
answer:  don't get mad .
 model:  don't get mad .

 input:  진실은 언젠가 드러나는 거예요.
answer:  the truth will come out .
 model:  the truth will come out .

 input:  불쾌한 날씨죠?
answer:  nasty weather , isn't it ?
 model:  nasty weather , isn't it ?

 input:  화내지 마세요.
answer:  please don't lose your temper .
 model:  don't don't angry get temper .

correct:  96

Epoch #43
loss: 0.28099942
val_loss: 0.16527152
 input:  여기서 내려 주세요.
answer:  let me off here , please .
 model:  let me off here , please .

 input:  어디가는 길이세요?
answer:  where are you headed ?
 model:  where are you headed ?

 input:  진정하세요!
answer:  slow down !
 model:  simmer down !

 input:  뉴욕행 열차 보통석으로 왕복권을 사고 싶습니다.
answer:  i'd like to buy a second class , round-trip ticket to new york .
 model:  i'd like to buy a second class , round-trip to to new york .

 input:  아주 좋은 날씨야.
answer:  what a beautiful day !
 model:  what a nice day .

 input:  좀 도와 주시겠어요?
answer:  would you mind helping me ?
 model:  could you mind helping me ?

correct:  126

Epoch #46
loss: 0.24520324
val_loss: 0.12628034
 input:  물론이죠. 어서 타세요.
answer:  sure . get in , please .
 model:  sure . get in , please .

 input:  힘 내시고, 다음 번에 다시 해 보세요.
answer:  come on , try it again .
 model:  come on , the it again .

 input:  언제라도 전화하세요.
answer:  please call me any time .
 model:  please call me any time .

 input:  형편에 달렸겠지요.
answer:  that depends .
 model:  that depends .

 input:  불쾌한 날씨죠?
answer:  nasty weather , isn't it ?
 model:  nasty weather , isn't it ?

 input:  좀 천천히 해!
answer:  slow down a little !
 model:  slow down a little !

correct:  136

Epoch #47
loss: 0.23422107
val_loss: 0.115220316
 input:  은혜는 평생 잊지 않겠습니다
answer:  i shall never forget your kindness as long as i live .
 model:  i shall never forget your kindness as long as i live .

 input:  좋은 계절이 시작되는군요, 그렇죠?
answer:  we are beginning to enjoy lovely weather , aren't we ?
 model:  we are beginning to enjoy lovely weather , aren't we ?

 input:  가끔 전화 주세요. 
answer:  please phone me occasionally .
 model:  please phone me occasionally .

 input:  비가 오고 있나요?
answer:  is it raining now ?
 model:  is it raining now ?

 input:  모든 것을 다 말해 주세요.
answer:  talk about every aspect of it .
 model:  talk about every aspect of it .

correct:  148

Epoch #48
loss: 0.2054679
val_loss: 0.10596013
 input:  좋은 계절이 시작되는군요, 그렇죠?
answer:  we are beginning to enjoy lovely weather , aren't we ?
 model:  we are beginning to enjoy lovely weather , aren't we ?

 input:  센트럴 공원으로 나가는 출구가 어디입니까?
answer:  where is the exit for central park ?
 model:  where is the exit for central park ?

 input:  미스터 송은 가장 친한 친구중에 하나입니다.
answer:  mr . song is one of my best friends .
 model:  mr . song is one of my best friends .

 input:  싹이 돋아났어요.
answer:  the buds come out all green .
 model:  the buds come out all green .

 input:  30분마다 1달러씩 받습니다.
answer:  we charge one dollar for every thirty minutes .
 model:  we charge one dollar for every thirty minutes .

correct:  156

Epoch #52
loss: 0.20748475
val_loss: 0.08774358
 input:  그곳을 버스로 갈 수 있습니까?
answer:  can i get there by bus ?
 model:  can i get there by bus ?

 input:  그럼, 다음에 또 뵐게요. 안녕히 계세요.
answer:  well , see you later . good bye .
 model:  well , see you later . good bye .

 input:  아직 한창 나인데 세상을 떠나시다니 참 애석한 일입니다.
answer:  it's such a pity to lose him so young .
 model:  it's such a pity to lose him so young .

 input:  이젠 비가 안옵니다.
answer:  it is no longer raining .
 model:  it is no longer raining .

 input:  비자를 신청하려고 합니다.
answer:  i'd like to apply for a visa .
 model:  i'd like to apply for you visa .

correct:  165